# Plot Chosen Clusters

## Mount Drive

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Libraries

In [ ]:
%%capture
# geopandas install
import os
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
os.chdir('/content/spatialindex-src-1.8.5')
!./configure
!make
!make install
!ldconfig
!pip install descartes
!pip install rtree
!pip install geopandas

In [ ]:
# standard libraries
import numpy as np
import pandas as pd
import pickle

# geopandas import
import geopandas as gpd

# folium import
import folium
import branca.colormap as cm

# import classes
import sys
sys.path.append(root + 'Classes')
from Stability_class import Stability

## Functions for Plot

In [ ]:
def Overlay(df, style_function, highlight_function, name):
    """
    Makes plot of a clustering.
    """
    Plot = folium.features.GeoJson(
    df,
    name=name,
    style_function=style_function,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['label','GAM','num_MSOAs','num_trusts','pop'],
        aliases=['Cluster Label','GAM','Number of MSOAs','Number of Trusts',
                 'Population'],
        localize = True,
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    ))
    return Plot


def addmarkers(map, color='black'):
    """Adds hospital trust locations to map."""

    for i, row in Pro.iterrows():
        marker = folium.CircleMarker(location = [row.geometry.y,row.geometry.x],
                                    radius=3,
                                    color=color, 
                                    popup=row['NAME']).add_to(map)
        
        map.keep_in_front(marker)

def colormap(n):
        """ Finds the a categorical color map for cluster of size n.
            Input:
                  n: cluster size.
            Output: 
                colormap: Categorical color map for Folium plots.
        """
        #creates categorical colour map
        return cm.linear.Spectral_11.to_step(index=np.arange(-0.5,n))

# function which converts array to 4.s.f
def sf4(x):
    x = float(np.format_float_positional(x, precision=4, unique=False, fractional=False))
    return x

vec_sf4 = np.vectorize(sf4)

## Set Up

In [ ]:
stability_data = Stability(root + 'Stability Data/longrun.mat')

# Array of number of communities.
k = stability_data.k

# Array of Markov times
times = stability_data.t

## Make Plots

In [ ]:
# creates map
m = folium.Map([54.5,-3],zoom_start=6.8,tiles='cartodbpositron')

#title
title_html = '''
            <h3 align="center" style="font-size:16px"><b>{}</b></h3>
            '''.format('Geographic Adjacency Measure of Clusters Across Time.')  

#sets style of highlight                            
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0}

# MSOA baseplate
MSOAs = gpd.read_file(root + 'MSOAs/MSOAs.shp')
MSOAs_fg = folium.FeatureGroup(name='MSOA Highlighter',show=False)
# plot boundary
MSOA_layer = folium.GeoJson(
MSOAs,
style_function = lambda x: {
    'opacity':0,
    'fillOpacity': 0},
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['msoa11nm','pop'],
        aliases=['Name','Population'],
        localize = True,
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )).add_to(MSOAs_fg)
MSOAs_fg.add_to(m)
m.keep_in_front(MSOAs_fg)

times = vec_sf4(times)

#get overlays
dfs = []
scores = []
catmaps = []
my_times = [7.499e3,3.350e3,188.4,19.95,0.5623]

for time in my_times:
    # gets index 
    idx = int(np.where(times == float(time))[0][0])

    # number of communities
    n = k[idx]

    # colormaps
    catmap = colormap(n)
    catmaps.append(catmap)

    # load df and score
    cluster_stab_df = gpd.read_file(root  + 'Cluster_Visualisation/' + str(time) + '/cluster_stab_df.shp')
    with open(root  + 'Cluster_Visualisation/' + str(time) + '/GAM_score.pickle', 'rb') as f:
        score = pickle.load(f)
    dfs.append(cluster_stab_df)
    scores.append(score)

    b_fg = folium.FeatureGroup(name='Boundary at Time %.4g'%time,show=False)
    # plot boundary
    b = folium.GeoJson(
    cluster_stab_df,
    style_function = lambda x: {
        'color': 'black',
        'weight': 2.5,
        "opacity":1,
        'fillOpacity': 0,
        'interactive':False
    }).add_to(b_fg)
    b_fg.add_to(m)
    m.keep_in_front(b_fg)

style_function1 = lambda x: {"opacity":0.3,
                            'color': 'black',
                            "fillColor":catmaps[0](x['properties']['label']), 
                            'fillOpacity':1}
style_function2 = lambda x: {"opacity":0.3,
                            'color': 'black',
                            "fillColor":catmaps[1](x['properties']['label']), 
                            'fillOpacity':1}
style_function3 = lambda x: {"opacity":0.3,
                            'color': 'black',
                            "fillColor":catmaps[2](x['properties']['label']), 
                            'fillOpacity':1}
style_function4 = lambda x: {"opacity":0.3,
                            'color': 'black',
                            "fillColor":catmaps[3](x['properties']['label']), 
                            'fillOpacity':1}
style_function5 = lambda x: {"opacity":0.3,
                            'color': 'black',
                            "fillColor":catmaps[4](x['properties']['label']), 
                            'fillOpacity':1}

styles = [style_function1,style_function2,style_function3,style_function4,style_function5]

for i in range(len(catmaps)): 
    c_fg = folium.FeatureGroup(name='Clustering at Time %.4g' %(my_times[i]),show=False)
    # get overlay
    overlay = Overlay(dfs[i],
              styles[i], 
              highlight_function,
              '').add_to(c_fg)

    # adds result to figure
    c_fg.add_to(m)

#adds title to plot
m.get_root().html.add_child(folium.Element(title_html))

html="""
    <p style="text-align: center;font-size:14px;"><span style="font-family: Arial, Helvetica, sans-serif;"> <strong>GAM at Time %.4g</strong> &emsp; %.4f </span></p>
    <p style="text-align: center;font-size:14px;"><span style="font-family: Arial, Helvetica, sans-serif;"> <strong>GAM at Time %.4g</strong> &emsp; %.4f </span></p>
    <p style="text-align: center;font-size:14px;"><span style="font-family: Arial, Helvetica, sans-serif;"> <strong>GAM at Time %.4g</strong> &emsp; %.4f </span></p>
    <p style="text-align: center;font-size:14px;"><span style="font-family: Arial, Helvetica, sans-serif;"> <strong>GAM at Time %.4g</strong> &emsp; %.4f </span></p>
    <p style="text-align: center;font-size:14px;"><span style="font-family: Arial, Helvetica, sans-serif;"> <strong>GAM at Time %.4g</strong> &emsp; %.4f </span></p>
    """%(my_times[0],scores[0],my_times[1],scores[1],my_times[2],scores[2],my_times[3],scores[3],my_times[4],scores[4])
    

iframe = folium.IFrame(html=html, width=230, height=160)
popup = folium.Popup(iframe, max_width=3000)

Text = folium.Marker(location=[53.7,-4.3], popup=popup,
                     icon=folium.Icon(icon_color='white')
                     )
m.add_child(Text)

folium.LayerControl(collapsed=False).add_to(m)

Pro = gpd.read_file(root + 'MSOAs/PRO.shp')
addmarkers(m)

m.save(root + 'Plots/chosen_clusters.html')

m

Output hidden; open in https://colab.research.google.com to view.

In [41]:
# creates map
m = folium.Map([54.5,-3],zoom_start=6.8,tiles='cartodbpositron')

#title
title_html = '''
            <h3 align="center" style="font-size:16px"><b>{}</b></h3>
            '''.format('Geographic Adjacency Measure of Clusters Across Time.')  


den_style_function = lambda x: {"weight":0.2,
                            "color":'black',
                            "opacity":1,
                            "fillColor":den_colormap(np.log(x['properties']['pop_den'])), 
                            'fillOpacity':0.8}


# add population density to MSOAs
MSOAs['pop_den'] = (MSOAs['pop']*1000**2)/MSOAs['st_areasha']
MSOAs_fg = folium.FeatureGroup(name='Population Density',show=False)

# population density color map
#colormap = cm.linear.RdBu_11.colors
#colormap.reverse()
#den_colormap = cm.LinearColormap(colors=den_colormap)
#den_colormap = den_colormap.to_step(data=np.log(MSOAs['pop_den']), quantiles=np.linspace(0,1,20))
den_colormap = cm.linear.RdBu_11.to_step(data=np.log(MSOAs['pop_den']), quantiles=np.linspace(0,1,20))
den_colormap.caption = "Log Population Density"
# density baseplate
MSOA_layer = folium.GeoJson(
MSOAs,
style_function = den_style_function,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['msoa11nm','pop','pop_den'],
        aliases=['Name','Population','Population Density'],
        localize = True,
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )).add_to(MSOAs_fg)
MSOAs_fg.add_to(m)
m.keep_in_front(MSOAs_fg)

times = vec_sf4(times)

#get overlays
my_times = [7.499e3,3.350e3,188.4,19.95,0.5623]

for time in my_times:
    # gets index 
    idx = int(np.where(times == float(time))[0][0])

    # load df and score
    cluster_stab_df = gpd.read_file(root  + 'Cluster_Visualisation/' + str(time) + '/cluster_stab_df.shp')

    b_fg = folium.FeatureGroup(name='Boundary at Time %.4g'%time,show=False)
    # plot boundary
    b = folium.GeoJson(
    cluster_stab_df,
    style_function = lambda x: {
        'color': 'black',
        'weight': 7,
        "opacity":1,
        'fillOpacity': 0,
        'interactive':False
    }).add_to(b_fg)
    b_fg.add_to(m)
    m.keep_in_front(b_fg)

#adds title to plot
m.get_root().html.add_child(folium.Element(title_html))
m.add_child(den_colormap)
# layer control
m.add_child(folium.LayerControl('topleft', collapsed=False))
#m.save(root + 'Plots/density.html')

m

Output hidden; open in https://colab.research.google.com to view.